In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls

import json

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

import yaml

import utils

# show all columns of a pandas dataframe
pd.set_option('display.max_columns', None)

In [3]:
# Opening yaml files with relevant config for Plolty Chart Studio
# Change your personal credentials in this file
with open("config.yaml") as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)

# define the settings for chart studio
chart_studio.tools.set_credentials_file(username=cfg['user'],
                                        api_key=cfg['api_key'])

In [4]:
# Opening JSON file
f = open('data/wa_washington_zip_codes_geo.min.json')
 
# returns JSON object as 
# a dictionary
geo_wa = json.load(f)


In [5]:
# Define some attributes concerning plotting
pub_chart_studio = True

### Import & clean the data

The dataset shows Airbnb activity in Seattle, has been provided by Airbnb and can be downloaded on Kaggle: <br> 
https://www.kaggle.com/datasets/airbnb/seattle/data <br>

The dataset contains the following:
- Calendar, including listing id and the price and availability for that day (calender.csv)
- Listings, including full descriptions and average review score (listings.csv)
- Reviews, including unique id for each reviewer and detailed comments (reviews.csv)

In [6]:
# Import the datasets
df_cal = pd.read_csv("data/calendar.csv")
df_list = pd.read_csv("data/listings.csv")
df_rev = pd.read_csv("data/reviews.csv")

---
Check & modify the dataframe with the calendar

In [7]:
df_cal.head()

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


In [8]:
df_cal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393570 entries, 0 to 1393569
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   listing_id  1393570 non-null  int64 
 1   date        1393570 non-null  object
 2   available   1393570 non-null  object
 3   price       934542 non-null   object
dtypes: int64(1), object(3)
memory usage: 42.5+ MB


In [9]:
# change column date from str to datetime
df_cal['date'] = pd.to_datetime(df_cal.date)

# get the month
df_cal['month'] = df_cal['date'].apply(lambda x: x.month)

# get the month_name
df_cal['month_name'] = df_cal['date'].apply(lambda x: x.month_name())

# calculate the week
df_cal['week'] = df_cal['date'].apply(lambda x: x.isocalendar()[1])

# change column available from string to boolean
df_cal['available'] = df_cal['available'].apply(lambda x: True if x == 't' else False)

# calculate a new column without currency sign
df_cal['price_usd_cal'] = df_cal['price'].apply(lambda x: x[1:] if isinstance(x, str) else x)

# remove thousand separator and convert column to float
df_cal['price_usd_cal'] = df_cal['price_usd_cal'].apply(lambda x: x.replace(',', '') if isinstance(x, str) else x).astype(float)

# drop column price
df_cal = df_cal.drop(columns='price')

In [10]:
df_cal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393570 entries, 0 to 1393569
Data columns (total 7 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   listing_id     1393570 non-null  int64         
 1   date           1393570 non-null  datetime64[ns]
 2   available      1393570 non-null  bool          
 3   month          1393570 non-null  int64         
 4   month_name     1393570 non-null  object        
 5   week           1393570 non-null  int64         
 6   price_usd_cal  934542 non-null   float64       
dtypes: bool(1), datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 65.1+ MB


In [11]:
df_cal.describe()

,listing_id,date,month,week,price_usd_cal
count,1.393570e+06,1393570,1.393570e+06,1.393570e+06,934542.000000
mean,5.550111e+06,2016-07-04 00:00:00.000000256,6.528767e+00,2.643014e+01,137.944859
min,3.335000e+03,2016-01-04 00:00:00,1.000000e+00,1.000000e+00,10.000000
25%,3.258213e+06,2016-04-04 00:00:00,4.000000e+00,1.300000e+01,75.000000
50%,6.118244e+06,2016-07-04 00:00:00,7.000000e+00,2.600000e+01,109.000000
75%,8.035212e+06,2016-10-03 00:00:00,1.000000e+01,3.900000e+01,160.000000
max,1.034016e+07,2017-01-02 00:00:00,1.200000e+01,5.200000e+01,1650.000000
std,2.962274e+06,NaN,3.443855e+00,1.504692e+01,105.062870


In [12]:
df_cal.shape

(1393570, 7)

---
Check & modify the dataframe with the listings

In [13]:
df_list.head(3)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,NaN,NaN,NaN,NaN,https://a1.muscache.com/ac/pictures/67560560/c...,NaN,956883,https://www.airbnb.com/users/show/956883,Maija,2011-08-11,"Seattle, Washington, United States","I am an artist, interior designer, and run a s...",within a few hours,96%,100%,f,https://a0.muscache.com/ac/users/956883/profil...,https://a0.muscache.com/ac/users/956883/profil...,Queen Anne,3.0,3.0,"['email', 'phone', 'reviews', 'kba']",t,t,"Gilman Dr W, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,98119,Seattle,"Seattle, WA",US,United States,47.636289,-122.371025,t,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",NaN,$85.00,NaN,NaN,NaN,NaN,2,$5.00,1,365,4 weeks ago,t,14,41,71,346,2016-01-04,207,2011-11-01,2016-01-02,95.0,10.0,10.0,10.0,10.0,9.0,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",What's up with the free pillows? Our home was...,"Convenient bus stops are just down the block, ...",https://a0.muscache.com/ac/pictures/14409893/f...,https://a0.muscache.com/im/pictures/14409893/f...,https://a0.muscache.com/ac/pictures/14409893/f...,https://a0.muscache.com/ac/pictures/14409893/f...,5177328,https://www.airbnb.com/users/show/5177328,Andrea,2013-02-21,"Seattle, Washington, United States",Living east coast/left coast/overseas. Time i...,within an hour,98%,100%,t,https://a0.muscache.com/ac/users/5177328/profi...,https://a0.muscache.com/ac/users/5177328/profi...,Queen Anne,6.0,6.0,"['email', 'phone', 'facebook', 'linkedin', 're...",t,t,"7th Avenue West, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,98119,Seattle,"Seattle, WA",US,United States,47.639123,-122.365666,t,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",NaN,$150.00,"$1,000.00","$3,000.00",$100.00,$40.00,1,$0.00,2,90,today,t,13,13,16,291,2016-01-04,43,2013-08-19,2015-12-29,96.0,10.0,10.0,10.0,10.0,10.0,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazi

In [14]:
# rename the column id to listing_id for later join
df_list = df_list.rename(columns={'id' : 'listing_id'})

# change some columns from str to boolean
col_bool = ['host_is_superhost',
            'host_has_profile_pic',
            'host_identity_verified',
            'is_location_exact',
            'has_availability',
            'requires_license',
            'instant_bookable',
            'require_guest_profile_picture',
            'require_guest_phone_verification']

for col in col_bool:
    df_list[col] = df_list[col].apply(lambda x: True if x == 't' else False)


# change column from str to datetime
col_date = ['last_scraped', 'host_since', 'calendar_last_scraped', 'first_review', 'last_review']

for col in col_date:
    df_list[col] = pd.to_datetime(df_list[col])


# replace percentage sign and convert column from str to float
col_perc = ['host_response_rate', 'host_acceptance_rate']

for col in col_perc:
    df_list[col] = df_list[col].apply(lambda x: x[:-1] if isinstance(x, str) else x).astype(float)


# calculate a new column without currency sign
col_curr = ['price','extra_people', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee']

for col in col_curr:
    df_list[col] = df_list[col].apply(lambda x: x.replace(',', '') if isinstance(x,str) else x)
    df_list[col + '_usd'] = df_list[col].apply(lambda x: x[1:] if isinstance(x, str) else x).astype(float)

# generate numeric values for the column host_response_time
resp_dict = {
    'within an hour': 1,
    'within a few hours' : .75,
    'within a day' : .5,
    'a few days or more' : .25
}

df_list['host_response_time_val'] = df_list['host_response_time'].apply(lambda x: resp_dict[x] if isinstance(x, str) else x).astype(float)


# clean property_type
df_list['property_type'] = df_list['property_type'].apply(lambda x: x.replace(' ', '').replace('&', '_').replace('/', '_') if isinstance(x,str) else x)
df_list['property_type'] = df_list['property_type'].apply(lambda x: x.lower() if isinstance(x,str) else x)

temp_df_prop = pd.get_dummies(df_list['property_type'], dtype=int)

new_cols_prop = []
for entry in temp_df_prop.columns:
    new_cols_prop.append('property_type_' + entry)

temp_df_prop.columns = new_cols_prop


# clean zipcode
# replacement is okay, as zip code 98122 exists, see: https://www.unitedstateszipcodes.org/
df_list['zipcode'] = df_list['zipcode'].replace('99\n98122', '98122')
df_list = df_list.drop(index=df_list[df_list.zipcode.isna()].index)

temp_df_zip = pd.get_dummies(df_list['zipcode'], dtype=int)

new_cols_zip = []
for entry in temp_df_zip.columns:
    new_cols_zip.append('zipcode_' + entry)

temp_df_zip.columns = new_cols_zip

# Prepare and extend the column amenities
df_list['amenities'] = df_list['amenities'].apply(lambda x: x.replace('"', '').replace('{', '').replace('}', '').replace('/', '_').replace('(s)', ''))
df_list['amenities'] = df_list['amenities'].apply(lambda x: x.lower())
df_list['amenities'] = df_list['amenities'].apply(lambda x: x.replace(' ', '_'))

df_list['nr_amenities'] = df_list['amenities'].apply(lambda x: len(x.split(','))).astype(int)

temp_df_am = df_list['amenities'].str.get_dummies(sep=',')

new_cols_am = []
for entry in temp_df_am.columns:
    new_cols_am.append('amenity_' + entry)

temp_df_am.columns = new_cols_am

# concat dataframes
df_list = pd.concat([df_list, temp_df_prop, temp_df_zip, temp_df_am], axis = 1) 

In [15]:
# drop columns that are not further considered or needed
col_drop = ['thumbnail_url',
            'medium_url',
            'xl_picture_url',
            'host_location',
            'host_verifications',
            'state',
            'market',
            'smart_location',
            'country_code',
            'country',
            'jurisdiction_names',
            'price',
            'extra_people',
            'weekly_price',
            'monthly_price',
            'security_deposit',
            'cleaning_fee',
            'host_response_time',
            'amenities']

df_list = df_list.drop(columns=col_drop)

In [16]:
df_list['zipcode'].unique()

array(['98119', '98109', '98107', '98117', '98103', '98105', '98115',
       '98101', '98122', '98112', '98144', '98121', '98102', '98199',
       '98104', '98134', '98136', '98126', '98146', '98116', '98177',
       '98118', '98108', '98133', '98106', '98178', '98125', nan],
      dtype=object)

In [17]:
df_list.shape

(3818, 165)

---

In [18]:
df_rev.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...


---
Join the calendar and the listing data

In [19]:
df_merge = df_cal.join(df_list.set_index('listing_id'), on = 'listing_id', lsuffix='_cal', rsuffix='_list')

df_merge = df_merge[df_merge['available'] == True]

# calculate some values concernung prices 
df_merge['price_fluc'] = df_merge['price_usd_cal'] / df_merge['price_usd']
df_merge['price_p_accom'] = df_merge['price_usd'] / df_merge['accommodates']
df_merge['price_p_accom_cal'] = df_merge['price_usd_cal'] / df_merge['accommodates']
df_merge.head(2)

,listing_id,date,available,month,month_name,week,price_usd_cal,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,picture_url,host_id,host_url,host_name,host_since,host_about,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,zipcode,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,square_feet,guests_included,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,price_usd,extra_people_usd,weekly_price_usd,monthly_price_usd,security_deposit_usd,cleaning_fee_usd,host_response_time_val,nr_amenities,property_type_apartment,property_type_bed_breakfast,property_type_boat,property_type_bungalow,property_type_cabin,property_type_camper_rv,property_type_chalet,property_type_condominium,property_type_dorm,property_type_house,property_type_loft,property_type_other,property_type_tent,property_type_townhouse,property_type_treehouse,property_type_yurt,zipcode_98101,zipcode_98102,zipcode_98103,zipcode_98104,zipcode_98105,zipcode_98106,zipcode_98107,zipcode_98108,zipcode_98109,zipcode_98112,zipcode_98115,zipcode_98116,zipcode_98117,zipcode_98118,zipcode_98119,zipcode_98121,zipcode_98122,zipcode_98125,zipcode_98126,zipcode_98133,zipcode_98134,zipcode_98136,zipcode_98144,zipcode_98146,zipcode_98177,zipcode_98178,zipcode_98199,amenity_24-hour_check-in,amenity_air_conditioning,amenity_breakfast,amenity_buzzer_wireless_intercom,amenity_cable_tv,amenity_carbon_monoxide_detector,amenity_cat,amenity_dog,amenity_doorman,amenity_dryer,amenity_elevator_in_building,amenity_essentials,amenity_family_kid_friendly,amenity_fire_extinguisher,amenity_first_aid_kit,amenity_free_parking_on_premises,amenity_gym,amenity_hair_dryer,amenity_hangers,amenity_heating,amenity_hot_tub,amenity_indoor_fireplace,amenity_internet,amenity_iron,amenity_kitchen,amenity_laptop_friendly_workspace,amenity_lock_on_bedroom_door,amenity_other_pet,amenity_pets_allowed,amenity_pets_live_on_this_property,amenity_pool,amenity_safety_card,amenity_shampoo,amenity_smoke_detector,amenity_smoking_allowed,amenity_suitable_for_events,amenity_tv,amenity_washer,amenity_washer___dryer,amenity_wheelchair_accessible,amenity_wireless_internet,price_fluc,price_p_accom,price_p_accom_cal
0,241032,2016-01-04,True,1,January,1,85.0,https://www.airbnb.com/rooms/241032,2.016010e+13,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,NaN,NaN,https://a1.muscache.com/ac/pictures/67560560/c...,956883.0,https://www.airbnb.com/users/show/956883,Maija,2011-08-11,"I am an artist, interior designer, and run a s...",96.0,100.0,False,https://a0.muscache.com/ac/users/956883/profil...,https://a0.muscache.com/ac/users/956883/profil...,Queen Anne,3.0,3.0,True,True,"Gilman Dr W, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,98119,47.636289,-122.371025,True,apartment,Entire home/apt,4.0,1.0,1.0,1.0,Real Bed,NaN,2.0,1.0,365.0,4 weeks ago,True,14.0,41.0,71.0,346.0,2016-01-04,207.0,2011-11-01,2016-01-02,95.0,10.0,10.0,10.0,10.0,9.0,10.0,False,NaN,False,moderate,False,False,2.0,4.07,85.0,5.0,NaN,NaN,NaN,NaN,0.75,10.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

### Analyze the data

In the following the data shall be analyzed and following questions answered:
1) Which property types are available?
2) How is the price per accomodate distributed? 
3) Are there price fluctuation over the year?
4) Can a linear regression model be used to model the price per accomodate? 

In [20]:
df_list.head(2)

,listing_id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,picture_url,host_id,host_url,host_name,host_since,host_about,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,zipcode,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,square_feet,guests_included,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,price_usd,extra_people_usd,weekly_price_usd,monthly_price_usd,security_deposit_usd,cleaning_fee_usd,host_response_time_val,nr_amenities,property_type_apartment,property_type_bed_breakfast,property_type_boat,property_type_bungalow,property_type_cabin,property_type_camper_rv,property_type_chalet,property_type_condominium,property_type_dorm,property_type_house,property_type_loft,property_type_other,property_type_tent,property_type_townhouse,property_type_treehouse,property_type_yurt,zipcode_98101,zipcode_98102,zipcode_98103,zipcode_98104,zipcode_98105,zipcode_98106,zipcode_98107,zipcode_98108,zipcode_98109,zipcode_98112,zipcode_98115,zipcode_98116,zipcode_98117,zipcode_98118,zipcode_98119,zipcode_98121,zipcode_98122,zipcode_98125,zipcode_98126,zipcode_98133,zipcode_98134,zipcode_98136,zipcode_98144,zipcode_98146,zipcode_98177,zipcode_98178,zipcode_98199,amenity_24-hour_check-in,amenity_air_conditioning,amenity_breakfast,amenity_buzzer_wireless_intercom,amenity_cable_tv,amenity_carbon_monoxide_detector,amenity_cat,amenity_dog,amenity_doorman,amenity_dryer,amenity_elevator_in_building,amenity_essentials,amenity_family_kid_friendly,amenity_fire_extinguisher,amenity_first_aid_kit,amenity_free_parking_on_premises,amenity_gym,amenity_hair_dryer,amenity_hangers,amenity_heating,amenity_hot_tub,amenity_indoor_fireplace,amenity_internet,amenity_iron,amenity_kitchen,amenity_laptop_friendly_workspace,amenity_lock_on_bedroom_door,amenity_other_pet,amenity_pets_allowed,amenity_pets_live_on_this_property,amenity_pool,amenity_safety_card,amenity_shampoo,amenity_smoke_detector,amenity_smoking_allowed,amenity_suitable_for_events,amenity_tv,amenity_washer,amenity_washer___dryer,amenity_wheelchair_accessible,amenity_wireless_internet
0,241032.0,https://www.airbnb.com/rooms/241032,2.016010e+13,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,NaN,NaN,https://a1.muscache.com/ac/pictures/67560560/c...,956883.0,https://www.airbnb.com/users/show/956883,Maija,2011-08-11,"I am an artist, interior designer, and run a s...",96.0,100.0,False,https://a0.muscache.com/ac/users/956883/profil...,https://a0.muscache.com/ac/users/956883/profil...,Queen Anne,3.0,3.0,True,True,"Gilman Dr W, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,98119,47.636289,-122.371025,True,apartment,Entire home/apt,4.0,1.0,1.0,1.0,Real Bed,NaN,2.0,1.0,365.0,4 weeks ago,True,14.0,41.0,71.0,346.0,2016-01-04,207.0,2011-11-01,2016-01-02,95.0,10.0,10.0,10.0,10.0,9.0,10.0,False,NaN,False,moderate,False,False,2.0,4.07,85.0,5.0,NaN,NaN,NaN,NaN,0.75,10.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0

#### 1) Property types

In [21]:
# get the distribution for the property types
df_list_prop = df_list.property_type.value_counts().reset_index()
df_list_prop['rel']  = (df_list_prop['count'] / len(df_list) * 100).round(2)
df_list_prop['rel_cumsum'] = df_list_prop['rel'].cumsum()

nr_objs = df_list_prop['count'].sum()

# create the figure
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(x=df_list_prop['property_type'], y=df_list_prop['rel'], 
                     name = 'Relative Number [%]'
             ))

fig.add_trace(go.Scatter(x=df_list_prop['property_type'], y=df_list_prop['rel_cumsum'],
                        name = 'Cumulative Sum [%]',
                        mode = 'lines'),
                        secondary_y=True)


fig.update_layout(
    title='Distribution of the property types',
    title_font = {"size": 20},
    xaxis_title = 'Property type',
    template='plotly_dark',
    hovermode = 'x',
    width = 1000,
    height = 600,
    yaxis = dict(
        title=dict(text="Relative Number [%]"),
        side="left",
        range=[0, 50]),
    yaxis2 = dict(
        title=dict(text="Cumulative Sum [%]"),
        side="right",
        range=[0, 100],
        overlaying="y",
        tickmode="sync")
)

fig.update_yaxes(
    title_font = {"size": 16},
    tickfont = {"size": 12}
    )

fig.update_xaxes(
    title_font = {"size": 16},
    tickfont = {"size": 12}
    )

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1,
    xanchor="left",
    x=0
    ))


fig.add_annotation(
    xref="x domain",
    yref="y domain",
    x=.98,
    y=.9,
    text=f"Total number propertys: {nr_objs}",
    align="right",
    showarrow=False
)


fig.show()

In [22]:
fig.write_html('res_property_types.html')

In [23]:
if pub_chart_studio:
    py.plot(fig, filename="seattle_distr_prop_types", auto_open = True)

#### 2) Distribution of the price per accommomdate

In [24]:
df_merge.head(2)

,listing_id,date,available,month,month_name,week,price_usd_cal,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,picture_url,host_id,host_url,host_name,host_since,host_about,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,zipcode,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,square_feet,guests_included,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,price_usd,extra_people_usd,weekly_price_usd,monthly_price_usd,security_deposit_usd,cleaning_fee_usd,host_response_time_val,nr_amenities,property_type_apartment,property_type_bed_breakfast,property_type_boat,property_type_bungalow,property_type_cabin,property_type_camper_rv,property_type_chalet,property_type_condominium,property_type_dorm,property_type_house,property_type_loft,property_type_other,property_type_tent,property_type_townhouse,property_type_treehouse,property_type_yurt,zipcode_98101,zipcode_98102,zipcode_98103,zipcode_98104,zipcode_98105,zipcode_98106,zipcode_98107,zipcode_98108,zipcode_98109,zipcode_98112,zipcode_98115,zipcode_98116,zipcode_98117,zipcode_98118,zipcode_98119,zipcode_98121,zipcode_98122,zipcode_98125,zipcode_98126,zipcode_98133,zipcode_98134,zipcode_98136,zipcode_98144,zipcode_98146,zipcode_98177,zipcode_98178,zipcode_98199,amenity_24-hour_check-in,amenity_air_conditioning,amenity_breakfast,amenity_buzzer_wireless_intercom,amenity_cable_tv,amenity_carbon_monoxide_detector,amenity_cat,amenity_dog,amenity_doorman,amenity_dryer,amenity_elevator_in_building,amenity_essentials,amenity_family_kid_friendly,amenity_fire_extinguisher,amenity_first_aid_kit,amenity_free_parking_on_premises,amenity_gym,amenity_hair_dryer,amenity_hangers,amenity_heating,amenity_hot_tub,amenity_indoor_fireplace,amenity_internet,amenity_iron,amenity_kitchen,amenity_laptop_friendly_workspace,amenity_lock_on_bedroom_door,amenity_other_pet,amenity_pets_allowed,amenity_pets_live_on_this_property,amenity_pool,amenity_safety_card,amenity_shampoo,amenity_smoke_detector,amenity_smoking_allowed,amenity_suitable_for_events,amenity_tv,amenity_washer,amenity_washer___dryer,amenity_wheelchair_accessible,amenity_wireless_internet,price_fluc,price_p_accom,price_p_accom_cal
0,241032,2016-01-04,True,1,January,1,85.0,https://www.airbnb.com/rooms/241032,2.016010e+13,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,NaN,NaN,https://a1.muscache.com/ac/pictures/67560560/c...,956883.0,https://www.airbnb.com/users/show/956883,Maija,2011-08-11,"I am an artist, interior designer, and run a s...",96.0,100.0,False,https://a0.muscache.com/ac/users/956883/profil...,https://a0.muscache.com/ac/users/956883/profil...,Queen Anne,3.0,3.0,True,True,"Gilman Dr W, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,98119,47.636289,-122.371025,True,apartment,Entire home/apt,4.0,1.0,1.0,1.0,Real Bed,NaN,2.0,1.0,365.0,4 weeks ago,True,14.0,41.0,71.0,346.0,2016-01-04,207.0,2011-11-01,2016-01-02,95.0,10.0,10.0,10.0,10.0,9.0,10.0,False,NaN,False,moderate,False,False,2.0,4.07,85.0,5.0,NaN,NaN,NaN,NaN,0.75,10.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [25]:
# get the relevant columns
rel_cols = ['listing_id',
            'date',
            'week',
            'month',
            'month_name',
            'available',
            'price_usd',
            'price_usd_cal',
            'price_p_accom',
            'price_p_accom_cal',
            'neighbourhood_group_cleansed',
            'neighbourhood',
            'zipcode',
            'accommodates']

df_merge_f = df_merge[rel_cols]

df_merge_f.head(2)

,listing_id,date,week,month,month_name,available,price_usd,price_usd_cal,price_p_accom,price_p_accom_cal,neighbourhood_group_cleansed,neighbourhood,zipcode,accommodates
0,241032,2016-01-04,1,1,January,True,85.0,85.0,21.25,21.25,Queen Anne,Queen Anne,98119,4.0
1,241032,2016-01-05,1,1,January,True,85.0,85.0,21.25,21.25,Queen Anne,Queen Anne,98119,4.0


In [26]:
# calculate the histogram for the price per accomodate
counts, bins = np.histogram(df_merge_f.price_p_accom_cal, bins=range(0, 200, 5))
bin_label = str(bins[:-1]) + ' ... ' + str(bins[1:])

# calculate the relative values
rels = np.round(counts / len(df_merge_f) * 100, 2)
rels_cumsum = rels.cumsum()

# get the max values
max_val = np.round(df_merge_f.price_p_accom_cal.max(), 2)

# create the labels
bin_label = []

for i in range(len(bins)):
    if i < len(bins)-1:
        bin_label.append(str(bins[i]) + ' ... ' + str(bins[i+1]))


# create the figure
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(x=bin_label, y=rels, 
                     name = 'Relative Number [%]'
             ))

fig.add_trace(go.Scatter(x=bin_label, y=rels_cumsum, 
                        name = 'Cumulative Sum [%]',
                        mode = 'lines'),
                        secondary_y=True)

fig.update_layout(
    title='Distribution of the prices per accommodate',
    title_font = {"size": 20},
    xaxis_title = 'Price per accommodate class [USD]',
    template='plotly_dark',
    hovermode = 'x',
    width = 1000,
    height = 600,
    yaxis = dict(
        title=dict(text="Relative Number [%]"),
        side="left",
        range=[0, 15]),
    yaxis2 = dict(
        title=dict(text="Cumulative Sum [%]"),
        side="right",
        range=[0, 150],
        overlaying="y",
        tickmode="sync")
)

fig.update_yaxes(
    title_font = {"size": 16},
    tickfont = {"size": 12}
    )

fig.update_xaxes(
    title_font = {"size": 16},
    tickfont = {"size": 12}
    )

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1,
    xanchor="left",
    x=0
    ))


fig.add_annotation(
    xref="x domain",
    yref="y domain",
    x=.98,
    y=.9,
    text=f"Maximum Value: {max_val} USD/accommodate",
    align="right",
    showarrow=False
)


fig.show()

In [27]:
if pub_chart_studio:
    py.plot(fig, filename="seattle_distr_pric_accom", auto_open = True)

#### Median price per accommodate per zip code

In [28]:
# Function to calculate qunatiles in a variable way within an aggregtation
# Inspiration and adaption from the code here: https://stackoverflow.com/questions/17578115/pass-percentiles-to-pandas-agg-function

def quant(q):
    def calc_q(x):
        return x.quantile(q)
    return calc_q

In [29]:
# calculate aggregated values by zipcode
df_agg_zip = df_merge_f.groupby(['zipcode']).agg(
    median = ('price_p_accom_cal', 'median'),
    min = ('price_p_accom_cal', 'min'),
    max = ('price_p_accom_cal', 'max'),
    q01 = ('price_p_accom_cal', quant(0.01)),
    q05 = ('price_p_accom_cal', quant(0.05)),
    q95 = ('price_p_accom_cal', quant(0.95)),
    q99 = ('price_p_accom_cal', quant(0.99)),
    ngbhs = ('neighbourhood_group_cleansed', set)
).round(1).reset_index()

df_agg_zip.ngbhs = df_agg_zip.ngbhs.apply(list)
df_agg_zip.head(2)

,zipcode,median,min,max,q01,q05,q95,q99,ngbhs
0,98101,49.7,4.8,366.5,10.8,24.8,100.0,150.0,"[Cascade, Capitol Hill, Downtown]"
1,98102,45.0,6.5,197.0,17.3,23.2,77.5,110.0,"[Cascade, Capitol Hill, Downtown]"


##### Line Plot

In [30]:
fig = go.Figure([
    go.Scatter(
        name='Minimum',
        x=df_agg_zip['zipcode'],
        y=df_agg_zip['min'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=1),
        showlegend=False
    ),
    go.Scatter(
        name='Maximum',
        x=df_agg_zip['zipcode'],
        y=df_agg_zip['max'],
        marker=dict(color="#444"),
        line=dict(width=1),
        mode='lines',
        fillcolor='rgba(68, 68, 68, .1)',
        fill='tonexty',
        showlegend=False
    )
])

lines = ['q01', 'q05', 'median', 'q95', 'q99']
line_dict = {
    'q01' : '1% Quantile',
    'q05' : '5% Quantile',
    'median' : 'Median',
    'q95' : '95% Quantile',
    'q99' : '99% Quantile'
    }

for l in lines:
    fig.add_trace(
        go.Scatter(
            name=line_dict[l],
            x=df_agg_zip['zipcode'],
            y=df_agg_zip[l],
            line=dict(width=1.5),
            mode='lines',
            showlegend=True
        ),
    )


fig.update_layout(
    yaxis_title='Price per accommodate [USD]',
    xaxis_title = 'Zip code',
    title='Price per accommodate per zip code',
    title_font = {"size": 20},
    hovermode="x",
    template='plotly_dark',
    width = 1200,
    height = 600
)

fig.update_yaxes(
    title_font = {"size": 16},
    tickfont = {"size": 12}
    )

fig.update_xaxes(
    title_font = {"size": 16},
    tickfont = {"size": 12}
    )

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1,
    xanchor="left",
    x=0
))

# fig.update_xaxes(tick0 = 0, dtick = 4)

fig.show()

In [31]:
if pub_chart_studio:
    py.plot(fig, filename="seattle_price_zipcode", auto_open = True)

##### Choropleth

In [ ]:
# get the min and max values
min_val = np.floor(df_agg_zip['median'].min()/10)*10
max_val = np.ceil(df_agg_zip['median'].max()/10)*10

# create the figure
fig = px.choropleth(data_frame=df_agg_zip, geojson=geo_wa, locations='zipcode', featureidkey = 'properties.ZCTA5CE10',
                        color='median',
                        color_continuous_scale="Viridis",
                        scope="usa", 
                        range_color = [min_val, max_val],
                        custom_data=['ngbhs'],
                        labels={
                            'zipcode' : 'Zip code',
                            'median':'Median price [USD]',
                            'custom_data[0]' : 'Neighbourhoods'
                            }
                          )

fig.update_layout(
    title='Median prices per accommodate per zip code',
    title_font = {"size": 20},
    width = 1000,
    height = 600,
    template='plotly_dark'
)

fig.update_traces(
    hovertemplate="<br>".join([
        "Zip code: %{location}",
        "Median price [USD]: %{z}",
        "Neighbourhoods: %{customdata[0]}"
    ])
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0, "l":0,"b":0})
fig.show()

### 3) Fluctution of the median price over the year

#### Median price per accommodate per zipcode per month

In [33]:
# calculate prices per month and zipcode
df_agg_zip_month = df_merge_f.groupby(['month', 'month_name', 'zipcode']).agg(
    mean = ('price_p_accom_cal', 'mean'),
    median = ('price_p_accom_cal', 'median'),
    min_val = ('price_p_accom_cal', 'min'),
    max_val = ('price_p_accom_cal', 'max'),
    ngbhs = ('neighbourhood_group_cleansed', set)
).reset_index()

df_agg_zip_month.head(2)

,month,month_name,zipcode,mean,median,min_val,max_val,ngbhs
0,1,January,98101,46.437382,41.666667,4.75,180.0,"{Cascade, Capitol Hill, Downtown}"
1,1,January,98102,45.762980,42.500000,6.50,197.0,"{Cascade, Capitol Hill, Downtown}"


In [34]:
plot_animation = False

if plot_animation:

    min_val = np.floor(df_agg_zip_month['median'].min()/10)*10
    max_val = np.ceil(df_agg_zip_month['median'].max()/10)*10

    fig = px.choropleth(data_frame=df_agg_zip_month, geojson=geo_wa, locations='zipcode', featureidkey = 'properties.ZCTA5CE10',
                            color='median',
                            color_continuous_scale="Viridis",
                            scope="usa", 
                            range_color = [min_val, max_val],
                            labels={
                                'month_name' : 'Month',
                                'zipcode' : 'Zip code',
                                'median':'Median price per month [USD]'
                                },
                            animation_frame='month_name'
                            )

    fig.update_layout(
        title='Median prices per accommodate per zip code per month',
        title_font = {"size": 20},
        width = 1000,
        height = 600,
        template='plotly_dark'
    )


    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0, "l":0,"b":0})
    fig.show()

#### Relative Median price per zipcode per month

In [35]:
# join the data to be able to calculate the relative fluctuation
df_agg_merge = df_agg_zip_month.join(df_agg_zip[['zipcode', 'median']].set_index('zipcode'), on = 'zipcode', lsuffix='_month', rsuffix='_year')
df_agg_merge['rel'] = (df_agg_merge['median_month'] / df_agg_merge['median_year']).round(3)


df_agg_merge.head(5)

,month,month_name,zipcode,mean,median_month,min_val,max_val,ngbhs,median_year,rel
0,1,January,98101,46.437382,41.666667,4.750000,180.0,"{Cascade, Capitol Hill, Downtown}",49.7,0.838
1,1,January,98102,45.762980,42.500000,6.500000,197.0,"{Cascade, Capitol Hill, Downtown}",45.0,0.944
2,1,January,98103,38.289103,33.750000,12.125000,127.5,"{Northgate, Other neighborhoods, University Di...",38.5,0.877
3,1,January,98104,49.938905,45.000000,9.666667,250.0,"{Other neighborhoods, Downtown}",50.0,0.900
4,1,January,98105,36.840794,32.500000,7.250000,120.0,"{Downtown, Other neighborhoods, University Dis...",35.0,0.929


In [36]:
# pivot the data and rename the columns with the month name
df_pivot = df_agg_merge.pivot(index='month', columns='zipcode')['rel']

ordered_months = ["January", "February", "March", "April", "May", "June", 
      "July", "August", "September", "October", "November", "December"]

for i in df_pivot.index:
    df_pivot.rename(index={i : ordered_months[i-1]}, inplace=True)

In [37]:
# create the heatmap
fig = px.imshow(df_pivot, x=df_pivot.columns, y=df_pivot.index,
                color_continuous_scale=px.colors.diverging.balance,
                labels=dict(
                    x = 'Zipcode',
                    y = 'Month',
                    #color="Relative median price"
                    )
                )


fig.update_layout(
    title='Relative median price per accommodate per zip code per month',
    title_font = {"size": 20},
    xaxis_title = 'Zip code',
    yaxis_title = 'Month',
    width = 1200,
    height = 600,
    template='plotly_dark'
)

fig.update_layout(
    coloraxis=dict(colorbar=dict(
        len=.97)
                   ))

fig.update_yaxes(
    title_font = {"size": 16},
    tickfont = {"size": 12}
    )

fig.update_xaxes(
    title_font = {"size": 16},
    tickfont = {"size": 12}
    )

fig.update_layout(
        margin={"t": 70, "b": 70, "r": 0, "l": 0, "pad": 0},
    )

fig.show()

In [38]:
if pub_chart_studio:
    py.plot(fig, filename="seattle_heatmap_rel_price_zipcode", auto_open = True)

### 4) Can a linear regression model be used to model the price per accomodate? 

In [39]:
# get all the relevant columns that shall be considered for the modeling
rel_cols  = ['month',
            'accommodates',
            'number_of_reviews',
            'host_is_superhost',
            'bathrooms',
            'bedrooms',
            'price_p_accom_cal'
]

# use the columns from above
rel_cols = rel_cols + new_cols_prop + new_cols_zip + new_cols_am

df_merge_lm = df_merge[rel_cols]

# drop columns with missing data
df_merge_lm = df_merge_lm.drop(index=(df_merge_lm[df_merge_lm['price_p_accom_cal'].isna()].index))
df_merge_lm = df_merge_lm.drop(index=(df_merge_lm[df_merge_lm['bathrooms'].isna()].index))
df_merge_lm = df_merge_lm.drop(index=(df_merge_lm[df_merge_lm['bedrooms'].isna()].index))

df_merge_lm.head(2)

,month,accommodates,number_of_reviews,host_is_superhost,bathrooms,bedrooms,price_p_accom_cal,property_type_apartment,property_type_bed_breakfast,property_type_boat,property_type_bungalow,property_type_cabin,property_type_camper_rv,property_type_chalet,property_type_condominium,property_type_dorm,property_type_house,property_type_loft,property_type_other,property_type_tent,property_type_townhouse,property_type_treehouse,property_type_yurt,zipcode_98101,zipcode_98102,zipcode_98103,zipcode_98104,zipcode_98105,zipcode_98106,zipcode_98107,zipcode_98108,zipcode_98109,zipcode_98112,zipcode_98115,zipcode_98116,zipcode_98117,zipcode_98118,zipcode_98119,zipcode_98121,zipcode_98122,zipcode_98125,zipcode_98126,zipcode_98133,zipcode_98134,zipcode_98136,zipcode_98144,zipcode_98146,zipcode_98177,zipcode_98178,zipcode_98199,amenity_24-hour_check-in,amenity_air_conditioning,amenity_breakfast,amenity_buzzer_wireless_intercom,amenity_cable_tv,amenity_carbon_monoxide_detector,amenity_cat,amenity_dog,amenity_doorman,amenity_dryer,amenity_elevator_in_building,amenity_essentials,amenity_family_kid_friendly,amenity_fire_extinguisher,amenity_first_aid_kit,amenity_free_parking_on_premises,amenity_gym,amenity_hair_dryer,amenity_hangers,amenity_heating,amenity_hot_tub,amenity_indoor_fireplace,amenity_internet,amenity_iron,amenity_kitchen,amenity_laptop_friendly_workspace,amenity_lock_on_bedroom_door,amenity_other_pet,amenity_pets_allowed,amenity_pets_live_on_this_property,amenity_pool,amenity_safety_card,amenity_shampoo,amenity_smoke_detector,amenity_smoking_allowed,amenity_suitable_for_events,amenity_tv,amenity_washer,amenity_washer___dryer,amenity_wheelchair_accessible,amenity_wireless_internet
0,1,4.0,207.0,False,1.0,1.0,21.25,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
1,1,4.0,207.0,False,1.0,1.0,21.25,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0


Analysis of the influence of the numnber of features and the maximum price per accomodate

In [40]:
# cutoffs here pertains to the number of missing values allowed in the used columns.
# Therefore, lower values for the cutoff provides more predictors in the model.
# prices means the threshold for the maximum price per accomodate and reduces the dataset for the modeling
cutoffs = [1000000, 500000, 100000, 50000, 10000, 5000, 2500, 1000, 100, 50]
prices = [15, 20, 25, 30, 35, 40, 45, 50, 75, 100, 200, 300, 500]


df_res = utils.find_optimal_lm_mod(df_merge_lm, cutoffs, col_price='price_p_accom_cal', prices = prices)
df_res.head(2)

Caclculation started for cutoff 1000000
Caclculation started for cutoff 500000
Caclculation started for cutoff 100000
Caclculation started for cutoff 50000
Caclculation started for cutoff 10000
Caclculation started for cutoff 5000
Caclculation started for cutoff 2500
Caclculation started for cutoff 1000
Caclculation started for cutoff 100
Caclculation started for cutoff 50


,price,cutoff,num_features,r2_score_test,r2_score_train
0,15,1000000,5,0.126139,0.136422
1,20,1000000,5,0.157275,0.149327


In [41]:
# aggregate the data to eliminate duplicates for price and number of features
df_res_agg = df_res.groupby(['price', 'num_features']).agg(
    r2_test_mean = ('r2_score_test', 'mean'),
    r2_train_mean = ('r2_score_train', 'mean')
).round(3).reset_index()

df_res_agg

,price,num_features,r2_test_mean,r2_train_mean
0,15,5,0.126,0.136
1,15,18,0.614,0.635
2,15,39,0.793,0.810
3,15,47,0.854,0.861
4,15,73,0.925,0.935
...,...,...,...,...
112,500,73,0.261,0.257
113,500,77,0.263,0.259
114,500,80,0.263,0.259
115,500,84,0.267,0.263


In [42]:
# pivot the data for the plot later
df_res_pivot = df_res_agg.pivot(index = 'num_features', columns='price')['r2_test_mean']

# convert int to string to ensure equidistant spacing in the following plot
df_res_pivot.columns = df_res_pivot.columns.astype(str)
df_res_pivot.index = df_res_pivot.index.astype(str)

df_res_pivot

price,15,20,25,30,35,40,45,50,75,100,200,300,500
num_features,,,,,,,,,,,,,
5,0.126,0.157,0.080,0.087,0.093,0.095,0.097,0.100,0.088,0.090,0.098,0.092,0.091
18,0.614,0.299,0.132,0.105,0.102,0.101,0.108,0.115,0.113,0.117,0.130,0.120,0.119
39,0.793,0.402,0.194,0.147,0.154,0.145,0.162,0.177,0.194,0.208,0.222,0.208,0.207
47,0.854,0.428,0.202,0.160,0.168,0.155,0.175,0.189,0.211,0.219,0.230,0.215,0.214
73,0.925,0.509,0.276,0.221,0.208,0.203,0.226,0.242,0.266,0.273,0.280,0.262,0.261
77,0.924,0.519,0.298,0.232,0.211,0.205,0.226,0.242,0.266,0.274,0.281,0.263,0.263
80,0.926,0.531,0.298,0.239,0.220,0.215,0.232,0.247,0.268,0.275,0.281,0.263,0.263
84,0.926,0.531,0.298,0.243,0.222,0.217,0.233,0.247,0.269,0.280,0.285,0.267,0.267
90,0.925,0.541,0.316,0.251,0.232,0.223,0.237,0.250,0.270,0.280,0.285,0.267,0.267


In [43]:
# create the heatmap
fig = px.imshow(df_res_pivot, x=df_res_pivot.columns, y=df_res_pivot.index,
                color_continuous_scale='Viridis',
                labels=dict(
                    x = 'Maximum price per accomodate considered [USD]',
                    y = 'Number features considered [-]',
                    # color="R Square Test [-]"
                    )
                )


fig.update_layout(
    title='R square Test',
    title_font = {"size": 20},
    width = 1200,
    height = 600,
    template='plotly_dark'
)

fig.update_yaxes(
    title_font = {"size": 16},
    tickfont = {"size": 12}
    )

fig.update_xaxes(
    title_font = {"size": 16},
    tickfont = {"size": 12}
    )

fig.update_layout(
    coloraxis=dict(colorbar=dict(
        len=.97)
                   ))


fig.update_layout(
        margin={"t": 70, "b": 70, "r": 0, "l": 0, "pad": 0},
    )

fig.show()


In [44]:
if pub_chart_studio:
    py.plot(fig, filename="seattle_heatmap_rsquare_max_price_num_features", auto_open = True)